In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
from selenium.webdriver.common.keys import Keys
from unidecode import unidecode
from bs4 import BeautifulSoup
from Database import Laptops, session


driver = webdriver.Firefox(executable_path="C://geckodriver.exe")

def check_element_exist(mode, inp):
    try:
        return driver.find_element(mode, inp)
    except:
        return False

In [ ]:
def validate_pagination(pagination):
    try:
        num = pagination.text.split()[-1]
        return unidecode(num)
    except Exception as err:
        print(err)
        print("Valid pagination")
        return 100

def load_all_page(url):
    driver.get(url)
    body = driver.find_element("tag name", "html")
    pagination = check_element_exist("xpath", "/html/body/div[1]/div[1]/div[3]/div[3]/div[1]/div/section[1]/div[2]/div[2]/div[2]")
    while not pagination:
        print("[-] Not Found Element")
        body.send_keys(Keys.END)
        sleep(2)
        pagination = check_element_exist("xpath", "/html/body/div[1]/div[1]/div[3]/div[3]/div[1]/div/section[1]/div[2]/div[2]/div[2]")

    sleep(3)
    print("[+] Found Element")
    return validate_pagination(pagination)

In [ ]:

file = open("mobiles.csv", "a", encoding="utf-8")


def extract_products():
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    source = soup.find("div", {"class": "d-flex flex-wrap"})
    return source.find_all("div", {"class": "border-b border-l"})

def validate_link(link):
    valid = link.split("/")[:3]
    return "/".join(valid)

def extract_product_information(products):
    objects = []
    for product in products:
        link = validate_link(product.find("a")["href"])
        image = product.find("img")["data-src"]
        name = product.find("h2").text
        try:
            price = product.find(class_="d-flex ai-center jc-end gap-1 color-700 color-400 text-h5 grow-1").span.text
            file.write(f"{link}, {image}, {name}, {unidecode(price)},\n")
            objects.append(Laptops(link=link, photo=image, name=name, price=unidecode(price)))
        except:
            return False
    session.add_all(objects)
    session.commit()
    print("Add ", len(objects), "Objects in DataBase")


In [ ]:
core_url = "https://www.digikala.com/search/category-notebook-netbook-ultrabook/"
count = load_all_page(core_url)
pages = [i for i in range(1, int(count))]
cant = open("cant.txt", "a")

In [ ]:

for page in pages:
    url = core_url+"?page="+str(page)
    try:
        load_all_page(url=url)
        products = extract_products()
        output = extract_product_information(products)
        print("Extract Information", page)
        if output == False:
            print("Products Ended")
            break
    except Exception as err:
        cant.write(url+"\n")
        print(err ," on page",page)